<a href="https://colab.research.google.com/github/BautistaDavid/ColombianFootballLeagueStudies/blob/main/dimayor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install great_tables
import pandas as pd
# from great_tables import GT, style_color_gradient
from great_tables import GT, style_color_gradient


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.2/607.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 1.6 MB/s eta 0:00:00


ImportError: cannot import name 'style_color_gradient' from 'great_tables' (/usr/local/lib/python3.12/dist-packages/great_tables/__init__.py)

In [13]:
url_fixtures = 'https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/scores_fixtures/2025/resultados_dimayor_2025.csv'
url_final_rank = 'https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/final_ranking/2025-2/final_rank_202502.csv'


In [14]:

fixtures = pd.read_csv(url_fixtures)
fixtures.columns = fixtures.columns.str.lower()
fixtures = fixtures[['round','date','time','home','away','score','location']]
fixtures['home_goals'] = fixtures['score'].apply(lambda x: x.split('–')[0]).astype('int')
fixtures['away_goals'] = fixtures['score'].apply(lambda x: x.split('–')[1]).astype('int')
fixtures = fixtures[fixtures['round'].str.contains('Finalización')]

final_rank = pd.read_csv(url_final_rank)

In [17]:
temp_away = (
    fixtures
    .groupby('away')
    .agg(
        away_matches=('away_goals', 'count'),
        away_goals_scored=('away_goals', 'sum'),
        away_goals_conceded=('home_goals', 'sum')
    )
    .reset_index()
)

temp_away['ratio_away_goals_scored'] = temp_away['away_goals_scored'] / temp_away['away_matches']
temp_away['ratio_away_goals_conceded'] = temp_away['away_goals_conceded'] / temp_away['away_matches']

temp_home = (
    fixtures
    .groupby('home')
    .agg(
        home_matches=('home_goals', 'count'),
        home_goals_scored=('home_goals', 'sum'),
        home_goals_conceded=('away_goals', 'sum')
    )
    .reset_index()
)

temp_home['ratio_home_goals_scored'] = temp_home['home_goals_scored'] / temp_home['home_matches']
temp_home['ratio_home_goals_conceded'] = temp_home['home_goals_conceded'] / temp_home['home_matches']


temp_away = temp_away.rename(columns={'away': 'team'})
temp_home = temp_home.rename(columns={'home': 'team'})

temp = temp_home.merge(temp_away,on='team')
temp['total_matches'] = temp['home_matches'] + temp['away_matches']
temp = temp[[
    'team',
    'total_matches',
    'ratio_home_goals_scored',
    'ratio_away_goals_scored',
    'ratio_home_goals_conceded',
    'ratio_away_goals_conceded'
]]

temp.loc[temp['team'] == 'Rionegro','team'] = 'Águilas Doradas'
temp = temp.merge(final_rank,how='left',on='team').sort_values(by='rank_n')
round_cols = temp.drop(columns=
 ['team','total_matches','rank_label','rank_n']).columns

temp[round_cols] = temp[round_cols].round(2)

temp['rank_label'] = temp['rank_label'].str.replace(' Cuadrangular','')
temp['rank_label'] = temp['rank_label'].str.replace('Tabla General','')
temp['rank_label'] = temp['rank_label'].str.replace('Subcampeón','🥈')
temp['rank_label'] = temp['rank_label'].str.replace('Campeón','🥇')



temp.columns = ['Equipo','# Partidos','Ratio Goles \nLocal','Ratio Goles\n Visitante',
                'Ratio Goles \nRecibidos Local','Ratio Goles \nRecibidos Visitante',
                'Ranking','rank_n']
temp = temp[
    ['Ranking','Equipo','# Partidos','Ratio Goles \nLocal','Ratio Goles\n Visitante',
                'Ratio Goles \nRecibidos Local','Ratio Goles \nRecibidos Visitante']
]


In [24]:
temp_ = temp.query("Equipo != 'Afafasdlianza'")
# temp_["icon"] = temp["Equipo"].map({
#     "Alianza": '<img src="https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/escudos/Alianza.png" width="20">',
# })

def create_url_image(team):
  team = team.replace(' ','%20') + '.png'
  return f'<img src="https://raw.githubusercontent.com/BautistaDavid/ColombianFootballLeagueStudies/refs/heads/main/data/escudos/{team}" width="25">'

temp_[' '] = temp_['Equipo'].apply(create_url_image)
temp_ = temp_[[' '] + temp_.columns[:-1].tolist()]

In [34]:
temp_.columns

Index([' ', 'Ranking', 'Equipo', '# Partidos', 'Ratio Goles \nLocal',
       'Ratio Goles\n Visitante', 'Ratio Goles \nRecibidos Local',
       'Ratio Goles \nRecibidos Visitante'],
      dtype='object')

In [36]:
cols = ['Ratio Goles \nLocal',
       'Ratio Goles\n Visitante', 'Ratio Goles \nRecibidos Local',
       'Ratio Goles \nRecibidos Visitante']

vmin = temp_[cols].min().min()
vmax = temp_[cols].max().max()
GT(temp_).tab_header(title="S&P 500", subtitle=f" ").fmt_markdown(
    columns=" ").style_color_gradient(
        columns=cols,
        domain=[vmin, vmax],
        low="#f7fbff",
        high="#08306b"
    )


AttributeError: 'GT' object has no attribute 'style_color_gradient'

In [59]:
import polars as pl
import polars.selectors as cs
sel_rev = cs.starts_with("revenue")
sel_prof = cs.starts_with("profit")
sel_prof

cs.starts_with('profit')

In [60]:
from great_tables import GT
from great_tables.data import sp500


# Create a gt table based on the `sp500_mini` table data
(
    GT(sp500_mini)
    .tab_header(title="S&P 500", subtitle=f"{start_date} to {end_date}")
    .fmt_currency(columns=["open", "high", "low", "close"])
    .fmt_date(columns="date", date_style="wd_m_day_year")
    .fmt_number(columns="volume", compact=True)
    .cols_hide(columns="adj_close")
)

GT(_tbl_data=            date       open     high      low      close        volume  \
1398  2010-06-14  1095.0000  1105.91  1089.03  1089.6300  4.425830e+09   
1399  2010-06-11  1082.6500  1092.25  1077.12  1091.6000  4.059280e+09   
1400  2010-06-10  1058.7700  1087.85  1058.77  1086.8400  5.144780e+09   
1401  2010-06-09  1062.7500  1077.74  1052.25  1055.6899  5.983200e+09   
1402  2010-06-08  1050.8101  1063.15  1042.17  1062.0000  6.192750e+09   
1403  2010-06-07  1065.8400  1071.36  1049.86  1050.4700  5.467560e+09   

      adj_close  
1398  1089.6300  
1399  1091.6000  
1400  1086.8400  
1401  1055.6899  
1402  1062.0000  
1403  1050.4700  , _body=<great_tables._gt_data.Body object at 0x7aaac2861520>, _boxhead=Boxhead([ColInfo(var='date', type=<ColInfoTypeEnum.default: 1>, column_label='date', column_align='right', column_width=None), ColInfo(var='open', type=<ColInfoTypeEnum.default: 1>, column_label='open', column_align='right', column_width=None), ColInfo(var='high', type=<ColInfoTypeEnum.default: 1>, column_label='high', column_align='right', column_width=None), ColInfo(var='low', type=<ColInfoTypeEnum.default: 1>, column_label='low', column_align='right', column_width=None), ColInfo(var='close', type=<ColInfoTypeEnum.default: 1>, column_label='close', column_align='right', column_width=None), ColInfo(var='volume', type=<ColInfoTypeEnum.default: 1>, column_label='volume', column_align='right', column_width=None), ColInfo(var='adj_close', type=<ColInfoTypeEnum.hidden: 4>, column_label='adj_close', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7aaaf014b110>, _spanners=Spanners([]), _heading=Heading(title='S&P 500', subtitle='2010-06-07 to 2010-06-14', preheader=None), _stubhead=None, _summary_rows=<great_tables._gt_data.SummaryRows object at 0x7aaac0ec38f0>, _summary_rows_grand=<great_tables._gt_data.SummaryRows object at 0x7aaac0ec3890>, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7aaac0ec3680>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7aaac1a5cb60>, <great_tables._gt_data.FormatInfo object at 0x7aaac0ec35f0>, <great_tables._gt_data.FormatInfo object at 0x7aaac0ec35c0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', valu

In [69]:
temp

,Ranking,Equipo,# Partidos,Ratio Goles \nLocal,Ratio Goles\n Visitante,Ratio Goles \nRecibidos Local,Ratio Goles \nRecibidos Visitante
8,Campeón,Junior,28,2.07,1.29,0.79,1.43
18,Subcampeón,Tolima,28,1.36,1.07,0.79,0.79
4,2 A,CD América,26,1.54,0.85,0.92,1.15
3,2 B,CA Bucaramanga,26,1.62,1.08,0.69,1.00
12,3 A,Nacional,26,1.92,1.54,1.00,1.38
17,3 B,Santa Fe,26,1.23,0.85,0.69,1.00
7,4 A,Independiente,26,2.00,1.92,1.15,1.54
6,4 B,Fortaleza FC,26,1.15,1.00,0.77,1.46
1,9,Alianza,20,1.30,1.10,0.60,1.60
16,10,Águilas Doradas,20,1.70,1.20,1.30,1.40
